In [18]:
import pandas as pd
import geopandas as gp
import numpy as np
import os
import itertools
import datetime
from datetime import date

In [19]:
## initiate file paths
#screening_results_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/all_results_for_tableau.xlsx"
screening_results_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/all_results_joined_SL_20240103.csv"
sample_points_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/Sampling_Sites_MASTER.xlsx"
file_name = f"screening_results_join_geometry_{date.today().strftime('%Y%m%d')}"
scenario = 'Drinking Water'

In [20]:
sample_pts = pd.read_excel(sample_points_path, sheet_name = "Sample_Locations")
sample_pts_gdf = gp.GeoDataFrame(sample_pts, geometry=gp.points_from_xy(sample_pts.Longitude, sample_pts.Latitude), crs="EPSG:4326")

In [21]:
screening_results = pd.read_csv(screening_results_path)

In [22]:
try:
    screening_results['DATE'] = screening_results['DATE'].str[:10]
    screening_results["DATE"] = pd.to_datetime(screening_results["DATE"])
except:
    print("date not included")

In [23]:
sample_pts_gdf['Date']
sample_pts_gdf["Date"] = pd.to_datetime(sample_pts_gdf["Date"], format='mixed')

In [24]:
try:
    sample_pts_join_results = pd.merge(screening_results, sample_pts_gdf, left_on = ['Sample ID', 'DATE'], right_on = ['Sampling ID', 'Date'], how = 'left')
    sample_pts_join_results.rename(columns = {'Medium_x':'Medium'}, inplace = True)
except:
    sample_pts_join_results = pd.merge(screening_results, sample_pts_gdf, left_on = ['Sample ID'], right_on = ['Sampling ID'], how = 'left')
    sample_pts_join_results.rename(columns = {'Medium_x':'Medium'}, inplace = True)

In [25]:
sample_pts_join_results_gdf = gp.GeoDataFrame(sample_pts_join_results)
sample_pts_join_results_gdf['Date'] = sample_pts_join_results_gdf['Date'].astype(str)

In [26]:
sample_pts_join_results.to_csv(f"/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/{file_name}.csv")

In [27]:
try:
    exceedances_only = sample_pts_join_results[sample_pts_join_results['SL_exceeded']=='Y']
    exceedances_only.drop(columns=['Medium_y', 'DateTime', 'Issues Peter has Found','Resolved', 'Photo', 'Sampling ID', 'Site ID'], inplace = True)
except:
    exceedances_only = sample_pts_join_results_gdf
    print("no exceedance column")
#exceedances_only = exceedances_only[exceedances_only['Scenario']==scenario]
exceedances_only.to_csv(f"/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/{file_name}.csv", index=False)

no exceedance column


In [28]:
exceedances_only.columns

Index(['Medium', 'DATE', 'Sample ID', 'Chemical Group', 'Chemical',
       'Result Parameter Name', 'Scenario', 'Screening Level Type',
       'Screening Level Measurement', 'SL Unit', 'Result Value',
       'Result Value Units', 'Source', 'SL_exceeded', 'Result Data Qualifier',
       'SL_diff', 'Sampling ID', 'Site ID', 'DateTime', 'Date', 'Medium_y',
       'Latitude', 'Longitude', 'Description', 'Photo', 'geometry'],
      dtype='object')

In [29]:
'''scenarios = exceedances_only['Scenario'].unique()
for i in scenarios:
    df = exceedances_only[exceedances_only['Scenario']==i]
    df.to_csv(f"/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/screening results by scenario/screening_results_join_geometry_exceeded_{i}.csv", index=False)'''

'scenarios = exceedances_only[\'Scenario\'].unique()\nfor i in scenarios:\n    df = exceedances_only[exceedances_only[\'Scenario\']==i]\n    df.to_csv(f"/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/screening results by scenario/screening_results_join_geometry_exceeded_{i}.csv", index=False)'